In [1]:
!pip install flask-ngrok
!pip install flask
!pip install Flask-SQLAlchemy
!pip install bcrypt

     |████████████████████████████████| 63 kB 1.6 MB/s 


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir('/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server')

In [4]:
from flask import Flask, jsonify
import pickle
import os
import cv2
import numpy as np 
import datetime
import argparse
from model import *
from flask_sqlalchemy import SQLAlchemy
import os
import re
import random
import shutil
import time
import numpy as np
import cv2
import json
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 
from torchvision import transforms
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok
from moviepy.editor import *
from data.data_reader import DatasetReader
from data.data_splitter import DatasetSplit
from data.data_transformer import DatasetTransform
from data.transforms import SelectFrames, FrameDifference, Downsample, TileVideo, RandomCrop, Resize, RandomHorizontalFlip, Normalize, ToTensor

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2318336/45929032 bytes (5.0%)5472256/45929032 bytes (11.9%)8437760/45929032 bytes (18.4%)11739136/45929032 bytes (25.6%)15032320/45929032 bytes (32.7%)18104320/45929032 bytes (39.4%)21274624/45929032 bytes (46.3%)24420352/45929032 bytes (53.2%)27525120/45929032 bytes (59.9%)30769152/45929032 bytes (67.0%)34226176/45929032 bytes (74.5%)37601280/45929032 bytes (81.9%)

In [9]:
def convert_datetime(unixtime):
    """Convert unixtime to datetime"""
    date = datetime.datetime.fromtimestamp(unixtime).strftime('%Y-%m-%d %H_%M_%S')
    return date # format : str

def convert_unixtime(date_time):
    """Convert datetime to unixtime"""
    unixtime = datetime.datetime.strptime(date_time,'%Y-%m-%d %H_%M_%S').timestamp()
    return unixtime

In [10]:
model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/규원model_best.E_bi_max_pool_ALL_fold_0_t2021-08-23 16:09:11.tar'#'/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/model_best.E_bi_max_pool_ALL_fold_0_t2021-08-12 20:02:12.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/은서model_best.E_bi_max_pool_ALL_fold_0_t2021-08-23 16:48:09.tar'#'/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/model_best.E_bi_max_pool_ALL_fold_0_t2021-08-12 20:02:12.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/vgg19+bilstm_TestData.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/vgg19+lstm_TestData.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/vgg19+bilstm_CrawlingData.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/규민checkpoint.E_ALL_fold_0_t2021-08-22 00 06 06.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/규민/규민model_best.E_bi_max_pool_ALL_fold_0_t2021-08-25 14:38:06.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/호준/호준checkpoint.E_bi_max_pool_ALL_fold_0_t2021-08-25 14:36:29.tar'
#model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/호준/호준model_best.E_bi_max_pool_ALL_fold_0_t2021-08-25 14:36:29.tar'
model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/호준/호준model_best.E_bi_max_pool_ALL_fold_0_t2021-08-27 01:04:46.tar'

def network_factory(arch):
    if arch == 'E':
        from networks.E import VP
    elif arch == 'E_bi':
        from networks.E_bi import VP
    elif arch == 'E_bi_avg_pool':
        from networks.E_bi_avg_pool import VP
    elif arch == 'E_bi_max_pool':
        from networks.E_bi_max_pool import VP
    else:
        assert 0, "Bad network arch name: " + arch
    return VP

def read_video(filename):
    frames = []
    #print(filename)
    if not os.path.isfile(filename):
        print('file not found')
    cap = cv2.VideoCapture(filename)
    while cap.isOpened():
       ret, frame = cap.read()  
       if not ret:
           break
       frames.append(frame)
    cap.release()
    video = np.stack(frames)
    #print("printing vidoe shape : ", video.shape)
    return video

In [7]:
seed = 250
args = {
    'arch': 'E_bi_max_pool',
    'workers':1,
    'batch-size':1,
    'evalmodel':model_path,
    'kfold':0,
    'split':5,
    'frames':10,#5,#5,
    'lr':1e-6,
    'weight_decay':1e-1
}

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# create model
print("=> creating model '{}'".format(args['arch']))
VP = network_factory(args['arch'])

model = VP()
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), args['lr'],
                                weight_decay=args['weight_decay'])

# optionally resume from a checkpoint
if os.path.isfile(args['evalmodel']):
    print("=> loading checkpoint '{}'".format(args['evalmodel']))
    checkpoint = torch.load(args['evalmodel'])
    args['start_epoch'] = checkpoint['epoch']
    print('start_epoch : ', args['start_epoch'])
    best_prec = checkpoint['best_prec']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
            .format(args['evalmodel'], checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(args['evalmodel']))

=> creating model 'E_bi_max_pool'


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

=> loading checkpoint '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/호준/호준model_best.E_bi_max_pool_ALL_fold_0_t2021-08-27 01:04:46.tar'
start_epoch :  43
=> loaded checkpoint '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Test Data/호준/호준model_best.E_bi_max_pool_ALL_fold_0_t2021-08-27 01:04:46.tar' (epoch 43)


In [8]:
def graph_cctv_stats(violence,uncertain,end_time,duration=3, year=2021):
    from matplotlib import pyplot as plt
    import matplotlib.patches as mpatches 
    plt.rcParams["figure.figsize"] = [10, 2]

    st_t = 0
    en_t = int(end_time)
    year = [year]

    for i in range(0,en_t,3):
        if i in violence:
            plt.barh(year, [duration], left=i, color="red")
        elif i in uncertain:
            plt.barh(year, [duration], left=i, color="yellow")
        else:
            plt.barh(year, [duration], left=i, color="green")

    plt.yticks([])
    plt.xticks([st_t]+violence+uncertain+[en_t])
    
    var_V = mpatches.Patch(color='red',label = 'violence')
    var_U = mpatches.Patch(color='yellow',label = 'uncertain')
    var_S = mpatches.Patch(color='green',label = 'safe')

    plt.legend(handles=[var_V,var_U,var_S], title="cctv statistics", loc=10, bbox_to_anchor=(0.5, 1.5),  ncol=3)
    plt.show()

In [11]:
def make_video_db(db, video_path, daycare_name, accuracy, status=0):
    daycare_center = DaycareCenter.query.filter_by(name=daycare_name).one()

    video = Video(
        detection_time = time.time() + 9 * 3600,
        name = os.path.basename(video_path),
        accuracy = round(accuracy,2),
        status = status,
        loc_id = daycare_center.loc_id,
        dc_id = daycare_center.id
        )

    db.session.add(video)
    db.session.commit()
    return video

In [12]:
def make_report_db(db, video_info):
    police_station = ['답십리지구대', '용신지구대', '청량리파출소', '제기파출소', '전농1파출소', '전농2파출소','장안1파출소', '장안2파출소', '이문지구대', '휘경파출소', '회기파출소']
    
    video = Video.query.filter_by(name=video_info.name).one()
    video.status = 1
    
    report_data = ReportList(
        time = time.time() + 9 * 3600, 
        police_name = np.random.choice(police_station, 1)[0],
        status = '출동 전',
        loc_id = video.loc_id,
        dc_id = video.dc_id,
        vid_id = video.id
    )
    db.session.add(report_data)
    db.session.commit()

    return report_data


In [ ]:
import re
import time
import hashlib
import datetime
import pandas as pd
import numpy as np
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok
from flask_sqlalchemy import SQLAlchemy
from flask import flash
from model import *

app = Flask(__name__)
app.secret_key = 'super secret key'
app.config['SESSION_TYPE'] = 'filesystem'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False 
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Web/child_abuse_detection_database.db'
db = SQLAlchemy(app)
run_with_ngrok(app)   #starts ngrok when the app is run

saved_videos = []
daycare_center_name = '동대문어린이집'
starting_time = 0
end_time = 0
violence_threshold = 90
uncertain_threshold = 80

# 영상 저장을 위한 변수
save_video_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/saved video/'
save_violence_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Web/static/violence/'
save_uncertain_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Web/static/uncertain/'

violence_files = []
uncertain_files = []
violence_time = []
uncertain_time = []

@app.route('/predict', methods=['POST'])
def prediction():
    if request.method == 'POST':
        global violence_files, violence_time, uncertain_files, uncertain_files, starting_time, end_time
        current_time = time.time()+9*60*60
        if starting_time == 0:
            starting_time = current_time
        end = time.time()

        file = request.files['file']
        frames = file.read()
        
        FILE_OUTPUT = daycare_center_name + '_' + str(time.strftime('%Y-%m-%d_%H_%M_%S %p', time.gmtime(current_time))) + '.mp4'
        saved_videos.append(FILE_OUTPUT)

        out_file = open(save_video_path + FILE_OUTPUT, "wb") 
        out_file.write(frames)
        out_file.close()

        video_path = save_video_path + FILE_OUTPUT
        
        video = read_video(video_path)
        val_dataset = DatasetReader(video)
        val_transformations = transforms.Compose([Resize(size=224), SelectFrames(num_frames=args['frames']), FrameDifference(dim=0), Normalize(), ToTensor()])

        val_dataset = DatasetTransform(val_dataset, val_transformations)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=False)

        model.eval()

        for i, (input) in enumerate(val_loader):
            input_var = torch.autograd.Variable(input, volatile=True)
            input_var = input_var.cuda()

            # compute output
            output_dict = model(input_var)
            output = output_dict['classification']

            model_ret = output_dict['classification'].cpu().detach().numpy()[0][1] * 100
            if model_ret > violence_threshold:
                print('-----------------violence detected!!!!!!----------------')
                print(model_ret, '% violence detected')
                violence_files.append(FILE_OUTPUT)
                violence_time.append(current_time)
                shutil.copy(save_video_path+FILE_OUTPUT, save_violence_path+FILE_OUTPUT)
                save_name = save_violence_path+FILE_OUTPUT.split('.')[0] + '_' + str(round(model_ret,2)) +'.mp4'
                os.rename( save_violence_path+FILE_OUTPUT, save_name)

                # DB에 PUSH 하는부분
                video_info = make_video_db(db, save_name, daycare_center_name, model_ret, status=1)
                report_info = make_report_db(db,video_info)

            elif model_ret > uncertain_threshold:
                print('****** uncertainty detected ******')
                print(model_ret, '% violence detected')
                uncertain_files.append(FILE_OUTPUT)
                shutil.copy(save_video_path+FILE_OUTPUT, save_uncertain_path+FILE_OUTPUT)
                save_name = save_uncertain_path+FILE_OUTPUT.split('.')[0] + '_' + str(round(model_ret,2)) +'.mp4'
                os.rename( save_uncertain_path+FILE_OUTPUT,  save_name)

                # DB에 PUSH 하는부분
                video_info = make_video_db(db, save_name,daycare_center_name, model_ret)

            else:
                print('violence : ', round(model_ret,2),' %, - ',FILE_OUTPUT)
                os.rename(save_video_path+FILE_OUTPUT, save_video_path+FILE_OUTPUT.split('.')[0] + '_' + str(round(model_ret,2)) +'.mp4')
        print('model calulation time : ',round(time.time() - end,2), ' sec')
        end_time = current_time + 3

        if model_ret > violence_threshold:  
            return json.dumps(str(1))
        else:
            return json.dumps(str(0))

@app.route('/end_stream', methods=['GET'])
def end_stream():
    global violence_time, starting_time, end_time, uncertain_time
    violence = np.array(violence_time) - starting_time
    violence = violence.astype('uint32')
    uncertain = np.array(uncertain_time) - starting_time
    uncertain = uncertain.astype('uint32')
    endtime = end_time - starting_time
    graph_cctv_stats(violence,uncertain,endtime)

app.run()
# python .\run_cctv.py 3 "http://bb11-35-232-83-139.ngrok.io" "C:\Users\Seogki\GoogleDrive\데이터청년캠퍼스_고려대과정\child-abuse-detection\webcam live test\test_video17.mp4"

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bb11-35-232-83-139.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
-----------------violence detected!!!!!!----------------
92.22313165664673 % violence detected
-----------------violence detected!!!!!!----------------
90.49432873725891 % violence detected


127.0.0.1 - - [27/Aug/2021 11:50:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 11:50:23] "POST /predict HTTP/1.1" 200 -


****** uncertainty detected ******
82.25851058959961 % violence detected
model calulation time :  1.94  sec
model calulation time :  2.9  sec


...

model calulation time :  0.87  sec


[2021-08-27 11:50:38,686] ERROR in app: Exception on /end_stream [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-13-575806154e28>", line 120, in end_stream
    graph_cctv_stats(violence,uncertai

****** uncertainty detected ******
89.36094045639038 % violence detected
model calulation time :  0.78  sec


127.0.0.1 - - [27/Aug/2021 11:50:40] "POST /predict HTTP/1.1" 200 -


-----------------violence detected!!!!!!----------------
92.41628050804138 % violence detected
model calulation time :  0.8  sec


In [ ]:
'

In [ ]:
import numpy as np
def random_input(n=10):
    sample = list('비폭애')
    return np.random.choice(sample,n,[0.5,0.2,0.3])

In [ ]:
random_input(n=50) # 폭 - 폭력 데이터 / 비 - 비폭력 데이터 / 애 - 애매한 데이터

array(['폭', '애', '비', '폭', '애', '비', '비', '비', '폭', '비', '비', '비', '애',
       '비', '비', '폭', '애', '비', '폭', '비', '폭', '비', '폭', '폭', '비', '애',
       '비', '폭', '애', '애', '비', '애', '비', '폭', '폭', '애', '애', '애', '폭',
       '애', '폭', '애', '애', '폭', '비', '비', '폭', '애', '애', '비'], dtype='<U1')

In [ ]:
def create_vid(videos, status):
    if status == 1:
        print(videos, ' - violence folder')
    else:
        print(videos, ' - uncertain folder')

In [ ]:
flow = random_input(n=50)
print(flow)
prev = '비'
next = None

v_list = list()
u_list = list()

fflow = list()
v_status = 0
fflow.append(prev)

for next in flow:
    if next == '비':
        if prev != '비':
            fflow = ['비']
        else:
            create_vid(fflow,v_status)
            v_status = 0
        fflow = ['비']
    elif next == '폭':
        v_status = 1
        fflow.append('폭')
    elif next == '애':
        fflow.append('애')
    prev = next



# 비비 - [비]로 유지
# 비폭 - [비폭] - append - 시작
# 비애 - [비애] - append - 시작
# 폭비 - 영상 저장  => 초기화
# 폭폭 - [폭폭] - append - v
# 폭애 - [폭애] - append - v
# 애비 - 영상 저장 시점 - 폭력이 있다면 폭으로
# 애폭 - [애폭] - append - v
# 애애 - [애폭] - append

['애' '비' '폭' '폭' '애' '애' '폭' '비' '비' '폭' '비' '비' '애' '비' '비' '애' '비' '애'
 '비' '비' '비' '애' '비' '애' '비' '애' '폭' '애' '애' '폭' '폭' '애' '애' '폭' '비' '비'
 '애' '비' '폭' '비' '애' '애' '비' '애' '비' '비' '비' '폭' '폭' '애']
['비']  - violence folder
['비']  - violence folder
['비']  - uncertain folder
['비']  - uncertain folder
['비']  - uncertain folder
['비']  - violence folder
['비']  - violence folder
['비']  - uncertain folder
